In [1]:
try:
    #!rm foolsunet_imagenette.weights.h5
    passes += 1
    #!git stash
    !git pull --force
except:
    passes = 1
    !git clone https://github.com/arminwitte/FoolsUNet.git
    %cd FoolsUNet
    !git pull --force

Cloning into 'FoolsUNet'...
remote: Enumerating objects: 483, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 483 (delta 204), reused 213 (delta 139), pack-reused 155 (from 1)
Receiving objects: 100% (483/483), 110.61 MiB | 13.71 MiB/s, done.
Resolving deltas: 100% (285/285), done.
Updating files: 100% (41/41), done.
/kaggle/working/FoolsUNet
Already up to date.


In [2]:
import tensorflow as tf

print(tf.__version__)

2024-09-27 06:01:31.742157: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 06:01:31.742275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 06:01:31.988865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0


In [3]:
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
#tf.tpu.experimental.initialize_tpu_system(resolver)
#print("All devices: ", tf.config.list_logical_devices('TPU'))

In [4]:
import tensorflow_datasets as tfds

In [5]:
dataset, info = tfds.load("imagenette/320px-v2:1.*.*", with_info=True, data_dir="./data/")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/9469 [00:00<?, ? examples/s]

Shuffling data/imagenette/320px-v2/1.0.0.incomplete6BDJB8/imagenette-train.tfrecord*...:   0%|          | 0/94…

Generating validation examples...:   0%|          | 0/3925 [00:00<?, ? examples/s]

Shuffling data/imagenette/320px-v2/1.0.0.incomplete6BDJB8/imagenette-validation.tfrecord*...:   0%|          |…

Dataset imagenette downloaded and prepared to ./data/imagenette/320px-v2/1.0.0. Subsequent calls will reuse this data.


In [6]:
dataset

{Split('train'): <_PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>,
 Split('validation'): <_PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}

In [7]:
def normalize(input_image):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image

In [8]:
def load_image(datapoint):
    input_image = tf.image.resize(datapoint["image"], (256, 256))
    input_label = datapoint["label"]

    input_image = normalize(input_image)

    return input_image, input_label

In [9]:
TRAIN_LENGTH = info.splits["train"].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = TRAIN_LENGTH
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
print(f"train length: {TRAIN_LENGTH}")
print(f"train length: {TRAIN_LENGTH / BATCH_SIZE}")
print(f"train length: {STEPS_PER_EPOCH}")

train length: 9469
train length: 147.953125
train length: 147


In [10]:
train_images = dataset["train"].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset["validation"].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

In [11]:
from tensorflow.keras import layers
class Augment(layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = layers.RandomFlip(mode="horizontal", seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        return inputs, labels

In [12]:
train_batches = (
    train_images.cache()
    .shuffle(BUFFER_SIZE)
    .repeat()
    .batch(BATCH_SIZE, drop_remainder=True)
    .map(Augment())
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

test_batches = test_images.batch(BATCH_SIZE)

## Get pre-trained encoder and attach classification head

In [13]:
from foolsunet.model import foolsunet, encoder, classification_head

In [14]:
num_classes = 10

In [15]:
strategy = tf.distribute.MirroredStrategy()
#strategy = tf.distribute.TPUStrategy(resolver)
with strategy.scope():
    # base_model_1 = tf.keras.models.load_model("foolsunet_imagenet_resized_6-9_epoch.keras")
    # base_model_2 = foolsunet(channel_attention="eca")
    # base_model_1.summary()
    enc = encoder()
    print(enc.output_shape)
    head = classification_head(num_classes, input_shape=enc.output_shape[1:])
    

(None, 16, 16, 54)


In [16]:
def assemble_model():
    model = tf.keras.Sequential([enc,head])
    model.build()
    return model 

### Define encoder

# Use the activations of these layers
layer_names = [
    'block_1_downsample',   # 64x64
    #'block_2_downsample',   # 32x32
]
base_model_layers = [base_model_2.get_layer(name) for name in layer_names]
layer_names = [
    'block_2_invres_downsample',   # 32x32
    'block_3_invres_downsample',   # 16x16
    # 'block_4_invres_downsample',  # 8x8
    # 'block_5_invres_downsample',      # 4x4
    # 'block_6_invres_downsample'
]
base_model_layers += [base_model_2.get_layer(name) for name in layer_names]

# # Create the feature extraction model
# down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

# down_stack.trainable = False

### Define classifier

def assemble_model():
    inputs = tf.keras.layers.Input(shape=[256, 256, 3])
    x = inputs
#     x = base_model.input
#     x = down_stack(inputs)
    for layer in base_model_layers:
#         layer.trainable = False
        x = layer(x)
    x = tf.keras.layers.Conv2D(1024, 1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Activation("relu6")(x)
    x = tf.keras.layers.Conv2D(num_classes, 1)(x)
    x = tf.keras.layers.GlobalAveragePooling2D(name="outputs")(x)

    
    return tf.keras.Model(inputs=inputs, outputs=x)

def assemble_model():
    inputs = tf.keras.layers.Input(shape=[256, 256, 3])
    x = inputs
#     x = base_model.input
#     x = down_stack(inputs)
    for layer in base_model_layers:
#         layer.trainable = False
        x = layer(x)
    x = tf.keras.layers.Conv2D(1024, 1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Activation("relu6")(x)
    x = tf.keras.layers.AveragePooling2D(32)(x)
    x = tf.keras.layers.Conv2D(num_classes, 1)(x)
    x = tf.keras.layers.Flatten(name="outputs")(x)

    
    return tf.keras.Model(inputs=inputs, outputs=x)

In [17]:
# classification_head = tf.keras.Sequential([
#     tf.keras.layers.Input([8,8,128]),
#     tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(num_classes, name="outputs"),])
# # classification_head.build()

In [18]:
# model = tf.keras.Sequential([base_model.get_layer(name) for name in layer_names] + [layers.Flatten(),
#   layers.Dense(128, activation='relu'),
#   layers.Dense(num_classes, name="outputs"),
#                                                                                    ])
# model.build()

In [19]:
with strategy.scope():
    model = assemble_model()
    opt = tf.keras.optimizers.Adam(learning_rate=0.005)
    model.compile(
    optimizer=opt,
    #Anything between 2 and `steps_per_epoch` could help here.
    #steps_per_execution = 50,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
    model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ functional_1 (Functional)       │ ?                      │       100,266 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ ?                      │       656,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 757,076 (2.89 MB)

 Trainable params: 753,420 (2.87 MB)

 Non-trainable params: 3,656 (14.28 KB)

In [20]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

In [21]:
EPOCHS = 20
VAL_SUBSPLITS = 1
VALIDATION_STEPS = info.splits["validation"].num_examples // BATCH_SIZE // VAL_SUBSPLITS

model_history = model.fit(
    train_batches,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    validation_data=test_batches,
    # callbacks=[DisplayCallback()]
    # callbacks=[tf.keras.callbacks.EarlyStopping(restore_best_weights=True, start_from_epoch=10)],
)

Epoch 1/20


2024-09-27 06:03:21.020734: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/sequential_1/functional_1_1/block_1_conv_0_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


147/147 ━━━━━━━━━━━━━━━━━━━━ 415s 3s/step - accuracy: 0.2882 - loss: 2.2170 - val_accuracy: 0.0994 - val_loss: 2.4372
Epoch 2/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 356s 2s/step - accuracy: 0.5567 - loss: 1.3501 - val_accuracy: 0.3333 - val_loss: 2.4468
Epoch 3/20


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


147/147 ━━━━━━━━━━━━━━━━━━━━ 374s 3s/step - accuracy: 0.6298 - loss: 1.1318 - val_accuracy: 0.4567 - val_loss: 1.9354
Epoch 4/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 353s 2s/step - accuracy: 0.6626 - loss: 1.0140 - val_accuracy: 0.4762 - val_loss: 1.3507
Epoch 5/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 375s 3s/step - accuracy: 0.7159 - loss: 0.8798 - val_accuracy: 0.5758 - val_loss: 1.4768
Epoch 6/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 356s 2s/step - accuracy: 0.7306 - loss: 0.8283 - val_accuracy: 0.5714 - val_loss: 1.3612
Epoch 7/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 371s 3s/step - accuracy: 0.7527 - loss: 0.7682 - val_accuracy: 0.5927 - val_loss: 1.4577
Epoch 8/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 355s 2s/step - accuracy: 0.7670 - loss: 0.7262 - val_accuracy: 0.8571 - val_loss: 0.5936
Epoch 9/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 374s 3s/step - accuracy: 0.7809 - loss: 0.6754 - val_accuracy: 0.6516 - val_loss: 1.2659
Epoch 10/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 356s 2s/step - accuracy: 0.7831 - loss: 0.6530 - val_accuracy: 0.71

In [22]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2, 2, figsize=(10, 10))

ax[0, 0].plot(model_history.history["accuracy"])
ax[0, 0].plot(model_history.history["val_accuracy"])
ax[0, 0].set_title("accuracy")
ax[0, 0].set_ylabel("accuracy")
ax[0, 0].set_xlabel("epoch")
ax[0, 0].set_ylim(0.0,1.0)
ax[0, 0].legend(["train", "val"], loc="lower left")


ax[0, 1].plot(model_history.history["loss"])
ax[0, 1].plot(model_history.history["val_loss"])
ax[0, 1].set_yscale("log")
ax[0, 1].set_title("loss")
ax[0, 1].set_ylabel("loss")
ax[0, 1].set_xlabel("epoch")
ax[0, 1].set_ylim(0.1,10.0)
ax[0, 1].legend(["train", "val"], loc="lower left")

# ax[1, 0].plot(model_history.history["regression_output_weighted_mse"])
# ax[1, 0].plot(model_history.history["val_regression_output_weighted_mse"])
# ax[1, 0].set_yscale("log")
# ax[1, 0].set_title("val_regression_output_weighted_mse")
# ax[1, 0].set_ylabel("accuracy")
# ax[1, 0].set_xlabel("epoch")
# ax[1, 0].legend(["train", "val"], loc="lower left")

fig.tight_layout()

In [23]:
# Save the entire model as a `.keras` zip archive.
model.save("foolsunet_imagenette.keras")

In [24]:

model.save_weights("foolsunet_imagenette.weights.h5")

In [25]:
new_model = tf.keras.models.load_model("foolsunet_imagenette.keras")

# Show the model architecture
new_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ functional_1 (Functional)       │ ?                      │       100,266 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ ?                      │       656,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,263,918 (8.64 MB)

 Trainable params: 753,420 (2.87 MB)

 Non-trainable params: 3,656 (14.28 KB)

 Optimizer params: 1,506,842 (5.75 MB)